In [1]:
import cv2, io, json, logging, math, os, sys, tempfile
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython.display import clear_output, display

In [2]:
sns.set()
%matplotlib inline

In [ ]:
%run Common.ipynb
%run CustomVision.ipynb

**Get Projects...**

In [ ]:
projects = get_projects(custom_vision_training_key, custom_vision_endpoint)
projects.sort(key=lambda project: project.name)

**Get Iterations** using [Iteration class](https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-vision-customvision/azure.cognitiveservices.vision.customvision.training.models.iteration(class)?view=azure-python) for each Project...

In [ ]:
for project in projects:
    project_id = project.id
    print('Project Id {0}'.format(project_id))
    print('Project name {0}'.format(project.name))
    
    iterations = get_iterations(project.id, custom_vision_training_key, custom_vision_endpoint)
    print('{0} Iterations...'.format(len(iterations)))
    print('\n')
    
    for iteration in iterations:
        print('Iteration Id {0}'.format(iteration.id))
        print('Status {0}'.format(iteration.status))
        print('Create {0}'.format(iteration.created))
        print('Last modified {0}'.format(iteration.last_modified))
        print('Publish name {0}'.format(iteration.publish_name))
        print('\n')

Set the Project Id to plot...

In [6]:
project_id = '<enter project id here>'

In [ ]:
iterations = get_iterations(project_id, custom_vision_training_key, custom_vision_endpoint)

In [ ]:
iterations.sort(reverse=False, key=lambda iteration: iteration.last_modified)

In [ ]:
# Per iteration...
df = pd.DataFrame(columns = ['average_precision', 'precision', 'recall', 'tagged_image_count'])

# Per tag...
tag = {}

threshold = 0.5
overlap_threshold = 0.3

for iteration in iterations:
    iteration_performance = get_iteration_performance(project_id, iteration.id, threshold, overlap_threshold, custom_vision_training_key, custom_vision_endpoint)

    tagged_image_count = get_tagged_image_count(project_id, iteration.id, None, custom_vision_training_key, custom_vision_endpoint)
    
    df = df.append({
        'average_precision': iteration_performance.average_precision,
        'precision': iteration_performance.precision,
        'recall': iteration_performance.recall,
        'tagged_image_count': tagged_image_count
    }, ignore_index=True)
    
    for tag_performance in iteration_performance.per_tag_performance:
        if tag_performance.name not in tag:
            tag[tag_performance.name] = pd.DataFrame(columns = ['average_precision', 'precision', 'recall', 'tagged_image_count', ]) # it was ['average_precision', 'precision', 'recall'] but average_precision and precision have the same values for tags -> CHECK 

        tagged_image_count = get_tagged_image_count(project_id, iteration.id, [tag_performance.id], custom_vision_training_key, custom_vision_endpoint)
        
        tag[tag_performance.name] = tag[tag_performance.name].append({
            'average_precision': tag_performance.average_precision,
            'precision': tag_performance.precision,
            'recall': tag_performance.recall,
            'tagged_image_count': tagged_image_count,
        }, ignore_index=True)

In [ ]:
sns.lineplot(data=df[['average_precision', 'precision', 'recall']])
plt.show()

In [ ]:
sns.lineplot(data=df[['tagged_image_count']])
plt.show()

In [ ]:
tag_count = len(list(tag))

plt.figure(figsize=(20,35))

for i in range(0, tag_count):
    tag_name = list(tag)[i]
    plt.subplot(math.ceil(tag_count / 4), 4, i + 1)
    sns.lineplot(data=tag[tag_name][['average_precision', 'precision', 'recall']]).set_title(tag_name)

plt.show()

In [ ]:
tag_count = len(list(tag))

plt.figure(figsize=(20, 35))

for i in range(0, tag_count):
    tag_name = list(tag)[i]
    plt.subplot(math.ceil(tag_count / 4), 4, i + 1)
    sns.lineplot(data=tag[tag_name][['tagged_image_count']]).set_title(tag_name)

plt.show()